In [99]:
import pickle
import os
import pandas as pd
import numpy as np

In [100]:
model_path = r"Z:\project\Seoul Bike Prediction\XGBoost_regressor_r2_0.927_v1.pkl"

model = pickle.load(open(model_path,"rb"))
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=0.9, colsample_bynode=None, colsample_bytree=0.9,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.091, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

Load SC

In [111]:
sc_dump_path = r"Z:\project\Seoul Bike Prediction\sc.pkl"
sc = pickle.load(open(sc_dump_path,"rb"))

In [112]:
sc

StandardScaler()

## Input Data

In [102]:
# Independent columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Rented Bike Count        8760 non-null   int64  
 1   Hour                     8760 non-null   int64  
 2   Temperature(°C)          8760 non-null   float64
 3   Humidity(%)              8760 non-null   int64  
 4   Wind speed (m/s)         8760 non-null   float64
 5   Visibility (10m)         8760 non-null   int64  
 6   Solar Radiation (MJ/m2)  8760 non-null   float64
 7   Rainfall(mm)             8760 non-null   float64
 8   Snowfall (cm)            8760 non-null   float64
 9   Holiday                  8760 non-null   int64  
 10  Functioning Day          8760 non-null   int64  
 11  Day                      8760 non-null   int64  
 12  Month                    8760 non-null   int64  
 13  Year                     8760 non-null   int64  
 14  Spring                  

In [103]:
date = "10/08/2024"
Hour = 21
Temperature = 2
Humidity = 25
Wind_speed = 1.1
Visibility = 1980
Solar_Radiation = 0.0
Rainfall = 0.0
Snowfall = 0.0
Seasons = "Winter"
Holiday = "No Holiday"
Functioning_Day = "Yes"

## Convert input data for perdiction

In [104]:
from datetime import datetime

Holiday_dic = {"No Holiday":0,"Holiday":1}
Functioning_Day_dic = {"Yes":1,"No":0}

def get_string_to_datetime(date):
    dt = datetime.strptime(date,"%d/%m/%Y")
    return({"day":dt.day, "month":dt.month, "year":dt.year, "weekday":dt.strftime("%A")})
str_to_date = get_string_to_datetime(date)
str_to_date

{'day': 10, 'month': 8, 'year': 2024, 'weekday': 'Saturday'}

In [105]:
user_input_list = [Hour, Temperature, Humidity, Wind_speed, Visibility, Solar_Radiation, Rainfall, 
                   Snowfall, Holiday_dic[Holiday], Functioning_Day_dic[Functioning_Day],
                   str_to_date["day"], str_to_date["month"], str_to_date["year"]]

features_name = [ 'Hour', 'Temperature(°C)', 'Humidity(%)','Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
                 'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day','Month', 'Year']
df_u_input = pd.DataFrame([user_input_list],columns=features_name)
df_u_input

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,21,2,25,1.1,1980,0.0,0.0,0.0,0,1,10,8,2024


In [106]:
def seasons_to_df(Seasons):
    season_list = ['Spring', 'Summer', 'Winter']
    seasons_np = np.zeros((1,len(season_list)))

    seasons_df = pd.DataFrame(seasons_np,columns=season_list,dtype=int)
    if Seasons in season_list:
        seasons_df[Seasons]=1
    return seasons_df
df_seasons = seasons_to_df(Seasons)
df_seasons

,Spring,Summer,Winter
0,0,0,1


In [107]:
def days_df(weekday):
    weekday_list = ['Monday', 'Saturday','Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    weekday_np = np.zeros((1,len(weekday_list)))
    
    weekday_df = pd.DataFrame(weekday_np,columns=weekday_list,dtype=int)
    if weekday in weekday_list:
        weekday_df[weekday]=1
    return weekday_df
df_days = days_df(str_to_date["weekday"])
df_days

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,1,0,0,0,0


In [108]:
df_for_pred = pd.concat([df_u_input,df_seasons,df_days],axis=1)
df_for_pred

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,21,2,25,1.1,1980,0.0,0.0,0.0,0,1,...,2024,0,0,1,0,1,0,0,0,0


In [109]:
sc_df_for_pred = sc.transform(df_for_pred)
sc_df_for_pred

array([[ 1.37440765, -0.91964452, -1.62406554, -0.60857988,  0.88993374,
        -0.65916799, -0.13401915, -0.17102636, -0.22996655,  0.18708879,
        -0.65085225,  0.43089543, 21.99373561, -0.58569899, -0.57778965,
         1.75950288, -0.39503684,  2.43270737, -0.42003701, -0.40559398,
        -0.41153889, -0.41272349]])

## Model For Prediction

In [110]:
print(f"Rented Bike Demand on date: {date} and Time:{Hour} is: {round(model.predict(sc_df_for_pred).tolist()[0])}")

Rented Bike Demand on date: 10/08/2024 and Time:21 is: 383
